# Artist Features Training

## Imports

In [ ]:
import pandas as pd
import numpy as np

import os
import sys
from pathlib import Path
from dotenv import load_dotenv
from typing import List
import matplotlib.pyplot as plt

load_dotenv()
DATA_PATH = Path(os.getenv("DATA_PATH"))

# only for .ipynb because relative imports don't work
root_path = (DATA_PATH.parent) 
os.chdir(str(root_path))
 
import src.training.pre_training as t
import src.training.postprocessing as pp
import src.training.plotting as p

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import plot_confusion_matrix

# sklearn imports
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# Preprocessing

## Load Data

In [ ]:
df = t.get_artist_df()

# scaling
max_followers = df["followers"].max()
df["followers"] = df["followers"].apply(lambda x: x / (max_followers / 100))
df["genre_name"] = df["genre_name"].apply(t.encode_genre)

## Split data: features X and target variable y

In [ ]:
X = df.values[:, :2]
y = df["popularity"].apply(t.binary_popularity)

print(X)
print(y)

## Over-/Undersampling

In [ ]:
# sampled and encoded popularity
X, y = RandomUnderSampler(random_state=42).fit_resample(X, y)
# X, y = RandomOverSampler(random_state=42).fit_resample(X, y)

In [ ]:
# Plot distritbution after over-/undersampling
fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(111)
ax.set_title("pop distr")
ax.set_xlabel("popularity")
ax.set_ylabel("count")

plt.bar(list(set(y)), pp.count_distribution(y))

print(pd.DataFrame(y).value_counts())

## Train/Test-Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(X_train.shape)
print(X_test.shape)

# Classification

In [ ]:
# store classifiers for later plotting
clf_list = []

## Gaussian Naive Bayes

In [ ]:
print("Gaussian Naive Bayes")
gaussian_clf = GaussianNB()

# fit the model
gaussian_clf.fit(X_train, y_train)
clf_list.append(gaussian_clf)

pp.print_metrics(gaussian_clf, X_test, y_test)

## SVM

In [ ]:
print("SVC")
svc_clf = SVC()

# fit the model
svc_clf.fit(X_train, y_train)
clf_list.append(svc_clf)

pp.print_metrics(svc_clf, X_test, y_test)

## Neural Network

In [ ]:
print("Neural Network")
nn_clf = MLPClassifier()

# fit the model
nn_clf.fit(X_train, y_train)
clf_list.append(nn_clf)

pp.print_metrics(nn_clf, X_test, y_test)

## K-Neighbours Classifier

In [ ]:
print("K-Neighbours Classifier")
knn_clf = KNeighborsClassifier()

# fit the model
knn_clf.fit(X_train, y_train)
clf_list.append(knn_clf)

pp.print_metrics(knn_clf, X_test, y_test)

## Decision Trees

In [ ]:
print("Decision Trees")
dt_clf = DecisionTreeClassifier()

# fit the model
dt_clf.fit(X_train, y_train)
clf_list.append(dt_clf)

pp.print_metrics(dt_clf, X_test, y_test)

## Random forest

In [ ]:
# use different number of trees in forest (comparing different hyperparameters)
forest_size = [10,20,50,100,250]

# set seed for random state to get compareable results in every execution (forest randomness)
np.random.seed(500)

for trees in forest_size:
    # set forest size
    print("Predicting with forest size " + str(trees))
    rf = RandomForestClassifier(n_estimators=trees)

    # fit the model
    rf.fit(X_train, y_train)
    clf_list.append(rf)

    pp.print_metrics(rf, X_test, y_test)
    print("--------\n")

## Voting Classifier

In [ ]:
from sklearn.ensemble import VotingClassifier

ens_clf = VotingClassifier(estimators=[
    ('gauss', GaussianNB()), ('knn', KNeighborsClassifier()), ('rf', RandomForestClassifier(n_estimators=1000))
], voting='hard')

ens_clf.fit(X_train, y_train)

pp.print_metrics(ens_clf, X_test, y_test)

# Model Evaluation

## Metrics + Confusion Matrices

In [ ]:
# generate list of plots for each clf: metrics, cf_matrix, cf_matrix_norm
p_list = p.generate_model_plots(X_test, y_test, clf_list)

## Save/display plots

In [ ]:
# params
save_plots = True
n_cols = 3
document_title = "Artists, undersampling, all features, artists with lyrics (1-5)"
document_folder = "artist" # lyrics, model, artist, all

# save/display plots as jpg
# p.plots_from_list(document_title, p_list, document_folder, cols=n_cols, save=save_plots)

## Store model

In [ ]:
pp.store_model_to_file(rf, "rf_size=100_prec=70", "artist")

## Plotting

In [ ]:
import matplotlib.pyplot as plt

plt.title("Dataset Artists V1 + unpredicted popularity")
plt.xlabel("popularity")
plt.ylabel("artist count")

plt.bar(list(set(y_test)), pp.count_distribution(y_test))
plt.show()

### Confusion Matrix

In [ ]:
# confusion matrix
fig, cax = plt.subplots(figsize=(16, 16)) # subplot for larger size
plot_confusion_matrix(estimator=knn_clf, X=X_test, y_true=y_test, cmap=plt.cm.Blues,normalize=None,values_format=".2f",ax=cax)

plt.show()

In [ ]:
# Confusion matrix scaled to [0,1]
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict
import itertools

# y_pred = rf.predict(X_test)
y_pred = cross_val_predict(nn_clf, X_test, y_test, cv=7)

cm = confusion_matrix(y_test, y_pred, normalize='true')

classes = ["0", "1"]
# classes = ["1", "2", "3", "4", "5"]

fig, ax = plt.subplots(1, 1)

im = ax.imshow(cm, vmin=0, vmax=1, cmap="Blues")
ax.set_title("Random forest on artist dataset")
tick_marks = np.arange(len(classes))
ax.set_xticks(tick_marks)
ax.set_xticklabels(classes)
ax.set_yticks(tick_marks)
ax.set_yticklabels(classes)

for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    text_color = "white" if cm[i, j] >= 0.7 else "black"
    ax.text(j, i, format(cm[i, j], '.2f'),
                horizontalalignment="center",
                color=text_color)

ax.set_ylabel('True label')
ax.set_xlabel('Predicted label')

sm = plt.cm.ScalarMappable(cmap="Blues", norm=plt.Normalize(vmin=0, vmax=1))
plt.colorbar(sm)

plt.show()